In [150]:
import pandas
import os

current_path = os.path.abspath('')
df1 = pandas.read_excel(current_path + '/../data/集思录.xlsx')
df2 = pandas.read_excel(current_path + '/../data/上市发行.xlsx')
df3 = pandas.read_excel(current_path + '/../data/发行结果.xlsx')
df4 = pandas.read_excel(current_path + '/../data/上市表现.xlsx')
df5 = pandas.read_excel(current_path + '/../data/正股价格.xlsx', converters={'股票代码':str})

df1['可转债代码'] = df1['证券代码'].fillna('000000.SH').map(lambda x:[c.strip() for c in x[1:-1].replace('\'','').split(',')][1])
df2['可转债代码'] = df2['转债代码'].fillna('000000.SH').map(lambda x:[c.strip() for c in x.split('.')][0])
df3['可转债代码'] = df3['代码'].fillna('000000.SH').map(lambda x:[c.strip() for c in x.split('.')][0])
df4['可转债代码'] = df4['代码'].fillna('000000.SH').map(lambda x:[c.strip() for c in x.split('.')][0])

df = pandas.merge(df1,df2, on='可转债代码', how='left')
df = pandas.merge(df,df3, on='可转债代码', how='left')
df = pandas.merge(df,df4, on='可转债代码', how='left')

for idx, row in df.iterrows():
    # 第一个是正股代码， 第二个是可转债代码
    symbol = [c.strip() for c in row['证券代码'][1:-1].replace('\'','').split(',')]
    name = [c.strip() for c in row['证券名称'][1:-1].replace('\'','').split(',')]
    # 可转债上市日期
    listing_date = row['上市公告日期']
    # 可转债配售股权登记日 (T-1日)
    registration = row['股权登记日']
    #print(symbol, name, listing_date, registration)
    # 网上发行日 (T日)
    day_t = row['网上发行日期_x'].strftime("%Y-%m-%d")
    # 发行公告日 (T-2日)
    day_t_2 = row['发行公告日'].strftime("%Y-%m-%d")

    print(symbol, name, day_t_2, registration, day_t, listing_date)
    price = df5[(df5['股票代码']==symbol[0]) & (df5['日期']==day_t)]
    #print(price)
    for idx in price.index:
        df.loc[df['可转债代码']==symbol[1], 'T日开盘'] = price.at[idx,'开盘']
        df.loc[df['可转债代码']==symbol[1], 'T日收盘'] = price.at[idx,'收盘']
        break
    price = df5[(df5['股票代码']==symbol[0]) & (df5['日期']==registration)]
    for idx in price.index:
        df.loc[df['可转债代码']==symbol[1], 'T-1日开盘'] = price.at[idx,'开盘']
        df.loc[df['可转债代码']==symbol[1], 'T-1日收盘'] = price.at[idx,'收盘']
        break
    price = df5[(df5['股票代码']==symbol[0]) & (df5['日期']==day_t_2)]
    for idx in price.index:
        df.loc[df['可转债代码']==symbol[1], 'T-2日开盘'] = price.at[idx,'开盘']
        df.loc[df['可转债代码']==symbol[1], 'T-2日收盘'] = price.at[idx,'收盘']
        break
        
    
df.to_excel(current_path + '/../data/data.xlsx', index=False)


['003043', '127079'] ['华亚智能', '华亚转债'] 2022-12-14 2022-12-15 2022-12-16 2023-01-16
['603176', '113665'] ['汇通集团', '汇通转债'] 2022-12-13 2022-12-14 2022-12-15 2023-01-11
['002645', '127077'] ['华宏科技', '华宏转债'] 2022-11-30 2022-12-01 2022-12-02 2023-01-10
['688350', '118029'] ['富淼科技', '富淼转债'] 2022-12-13 2022-12-14 2022-12-15 2023-01-09
['002998', '127078'] ['优彩资源', '优彩转债'] 2022-12-12 2022-12-13 2022-12-14 2023-01-09
['301017', '123172'] ['漱玉平民', '漱玉转债'] 2022-12-13 2022-12-14 2022-12-15 2023-01-06
['600761', '110091'] ['安徽合力', '合力转债'] 2022-12-09 2022-12-12 2022-12-13 2023-01-06
['688219', '118028'] ['会通股份', '会通转债'] 2022-12-02 2022-12-05 2022-12-06 2023-01-04
['603757', '113664'] ['大元泵业', '大元转债'] 2022-12-01 2022-12-02 2022-12-05 2023-01-04
['603809', '113662'] ['豪能股份', '豪能转债'] 2022-11-23 2022-11-24 2022-11-25 2022-12-23
['688066', '118027'] ['航天宏图', '宏图转债'] 2022-11-24 2022-11-25 2022-11-28 2022-12-22
['603806', '113661'] ['福斯特', '福22转债'] 2022-11-18 2022-11-21 2022-11-22 2022-12-22
['601665', '1130

['600438', '110085'] ['通威股份', '通22转债'] 2022-02-22 2022-02-23 2022-02-24 2022-03-18
['002381', '127054'] ['双箭股份', '双箭转债'] 2022-02-09 2022-02-10 2022-02-11 2022-03-15
['603585', '113640'] ['苏利股份', '苏利转债'] 2022-02-14 2022-02-15 2022-02-16 2022-03-10
['002988', '127053'] ['豪美新材', '豪美转债'] 2022-01-20 2022-01-21 2022-01-24 2022-03-04
['300763', '123137'] ['锦浪科技', '锦浪转债'] 2022-02-08 2022-02-09 2022-02-10 2022-03-02
['688116', '118005'] ['天奈科技', '天奈转债'] 2022-01-25 2022-01-26 2022-01-27 2022-03-01
['603186', '113639'] ['华正新材', '华正转债'] 2022-01-20 2022-01-21 2022-01-24 2022-02-22
['601012', '113053'] ['隆基绿能', '隆22转债'] 2021-12-31 2022-01-04 2022-01-05 2022-02-17
['300778', '123136'] ['新城市', '城市转债'] 2022-01-19 2022-01-20 2022-01-21 2022-02-15
['688166', '118004'] ['博瑞医药', '博瑞转债'] 2021-12-30 2021-12-31 2022-01-04 2022-01-27
['002534', '127052'] ['西子洁能', '杭锅转债'] 2021-12-22 2021-12-23 2021-12-24 2022-01-24
['603055', '113638'] ['台华新材', '台21转债'] 2021-12-27 2021-12-28 2021-12-29 2022-01-21
['300673', '12

['603650', '113621'] ['彤程新材', '彤程转债'] 2021-01-22 2021-01-25 2021-01-26 2021-02-22
['603920', '113619'] ['世运电路', '世运转债'] 2021-01-18 2021-01-19 2021-01-20 2021-02-19
['300651', '123093'] ['金陵体育', '金陵转债'] 2021-01-15 2021-01-18 2021-01-19 2021-02-18
['601229', '113042'] ['上海银行', '上银转债'] 2021-01-21 2021-01-22 2021-01-25 2021-02-10
['300065', '123086'] ['海兰信', '海兰转债'] 2020-12-09 2020-12-10 2020-12-11 2021-02-08
['603538', '113618'] ['美诺华', '美诺转债'] 2021-01-12 2021-01-13 2021-01-14 2021-02-04
['002931', '128143'] ['锋龙股份', '锋龙转债'] 2021-01-06 2021-01-07 2021-01-08 2021-01-29
['603501', '113616'] ['韦尔股份', '韦尔转债'] 2020-12-24 2020-12-25 2020-12-28 2021-01-22
['000552', '127027'] ['靖远煤电', '靖远转债'] 2020-12-08 2020-12-09 2020-12-10 2021-01-22
['002946', '128142'] ['新乳业', '新乳转债'] 2020-12-16 2020-12-17 2020-12-18 2021-01-19
['002034', '128141'] ['旺能环境', '旺能转债'] 2020-12-15 2020-12-16 2020-12-17 2021-01-18
['603707', '113614'] ['健友股份', '健20转债'] 2020-12-15 2020-12-16 2020-12-17 2021-01-18
['300332', '123092

['300511', '123056'] ['雪榕生物', '雪榕转债'] 2020-06-22 2020-06-23 2020-06-24 2020-07-24
['603687', '113591'] ['大胜达', '胜达转债'] 2020-06-25 2020-06-30 2020-07-01 2020-07-23
['002838', '128117'] ['道恩股份', '道恩转债'] 2020-06-30 2020-07-01 2020-07-02 2020-07-20
['603608', '113589'] ['天创时尚', '天创转债'] 2020-06-22 2020-06-23 2020-06-24 2020-07-16
['002444', '128115'] ['巨星科技', '巨星转债'] 2020-06-22 2020-06-23 2020-06-24 2020-07-16
['002157', '128114'] ['正邦科技', '正邦转债'] 2020-06-13 2020-06-16 2020-06-17 2020-07-15
['002832', '128113'] ['比音勒芬', '比音转债'] 2020-06-11 2020-06-12 2020-06-15 2020-07-15
['603039', '113587'] ['ST泛微', '泛微转债'] 2020-06-11 2020-06-12 2020-06-15 2020-07-14
['603108', '113588'] ['润达医疗', '润达转债'] 2020-06-15 2020-06-16 2020-06-17 2020-07-13
['300138', '123055'] ['晨光生物', '晨光转债'] 2020-06-15 2020-06-16 2020-06-17 2020-07-13
['002241', '128112'] ['歌尔股份', '歌尔转2'] 2020-06-10 2020-06-11 2020-06-12 2020-07-13
['002756', '128110'] ['永兴材料', '永兴转债'] 2020-06-05 2020-06-08 2020-06-09 2020-07-08
['002738', '12811

['300450', '123036'] ['先导智能', '先导转债'] 2019-12-07 2019-12-10 2019-12-11 2020-01-10
['603690', '113556'] ['至纯科技', '至纯转债'] 2019-12-18 2019-12-19 2019-12-20 2020-01-09
['002002', '128085'] ['鸿达兴业', '鸿达转债'] 2019-12-12 2019-12-13 2019-12-16 2020-01-08
['601615', '113029'] ['明阳智能', '明阳转债'] 2019-12-12 2019-12-13 2019-12-16 2020-01-07
['603180', '113553'] ['金牌厨柜', '金牌转债'] 2019-12-11 2019-12-12 2019-12-13 2020-01-07
['002806', '128082'] ['华锋股份', '华锋转债'] 2019-12-02 2019-12-03 2019-12-04 2020-01-06
['600567', '110063'] ['山鹰国际', '鹰19转债'] 2019-12-11 2019-12-12 2019-12-13 2020-01-03
['002376', '128083'] ['新北洋', '新北转债'] 2019-12-10 2019-12-11 2019-12-12 2019-12-31
['600498', '110062'] ['烽火通信', '烽火转债'] 2019-11-28 2019-11-29 2019-12-02 2019-12-25
['603960', '113552'] ['克来机电', '克来转债'] 2019-11-28 2019-11-29 2019-12-02 2019-12-19
['002203', '128081'] ['海亮股份', '海亮转债'] 2019-11-19 2019-11-20 2019-11-21 2019-12-16
['603035', '113550'] ['常熟汽饰', '常汽转债'] 2019-11-14 2019-11-15 2019-11-18 2019-12-12
['603806', '1135

['000001', '127010'] ['平安银行', '平银转债'] 2019-01-17 2019-01-18 2019-01-21 2019-02-18
['300381', '123018'] ['溢多利', '溢利转债'] 2018-12-18 2018-12-19 2018-12-20 2019-01-23
['002783', '128052'] ['凯龙股份', '凯龙转债'] 2018-12-19 2018-12-20 2018-12-21 2019-01-21
['600728', '110050'] ['佳都科技', '佳都转债'] 2018-12-17 2018-12-18 2018-12-19 2019-01-21
['600690', '110049'] ['海尔智家', '海尔转债'] 2018-12-14 2018-12-17 2018-12-18 2019-01-18
['603055', '113525'] ['台华新材', '台华转债'] 2018-12-13 2018-12-14 2018-12-17 2019-01-11
['002741', '128051'] ['光华科技', '光华转债'] 2018-12-12 2018-12-13 2018-12-14 2019-01-09
['603677', '113524'] ['奇精机械', '奇精转债'] 2018-12-12 2018-12-13 2018-12-14 2019-01-07
['600483', '110048'] ['福能股份', '福能转债'] 2018-12-05 2018-12-06 2018-12-07 2018-12-28
['002865', '128050'] ['钧达股份', '钧达转债'] 2018-12-06 2018-12-07 2018-12-10 2018-12-27
['603568', '113523'] ['伟明环保', '伟明转债'] 2018-12-06 2018-12-07 2018-12-10 2018-12-26
['000070', '127008'] ['特发信息', '特发转债'] 2018-11-14 2018-11-15 2018-11-16 2018-12-25
['300618', '12301